In [19]:
from future.utils import iteritems
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
import re
import pandas as pd
from sklearn.model_selection import train_test_split

In [20]:
import gensim.downloader

In [21]:
pci_vectors = pd.read_csv("pci_ce_100.csv")

In [22]:
w2v_model = gensim.downloader.load('word2vec-google-news-300')

In [301]:
train = pd.read_pickle("train.pkl")
test = pd.read_pickle("test.pkl")

In [302]:
train_x = train['token']
train_y = train['label']
train_id = train['sen_id']

In [303]:
test_x = test['token']
test_y = test['label']
test_id = test['sen_id']

In [304]:
vocab = set([item for sublist in train_x for item in sublist] + [item for sublist in test_x for item in sublist])
tag = ['O', 'C', 'E']

In [305]:
vectors =pci_vectors.set_index('token').T.to_dict('list')

<ipython-input-305-758d4c807898>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  vectors =pci_vectors.set_index('token').T.to_dict('list')


In [306]:
len(vectors['arrows'])

100

In [307]:
zero_300 = np.array([0] * 300)
zero_301 = np.concatenate((np.array([1]), zero_300))

zero_100 = np.array([0] * 100)
zero_101 = np.concatenate((np.array([1]), zero_100))


In [308]:
word2idx = {}

not_present = []

In [309]:
for i in vocab:
    try:
        vec = w2v_model[i]
        vec1 = np.concatenate((np.array([0]), vec))
        word2idx[i]= vec1
        
    except:
        word2idx[i]= zero_301


In [310]:
for i in vocab:
    try:
        vec = vectors[i]
        vec1 = np.concatenate((np.array([0]), vec))
        word2idx[i]= np.concatenate((word2idx[i], vec1))
    except:
        word2idx[i]= np.concatenate((word2idx[i], zero_101))
    

In [311]:
word2idx['dam']

array([ 0.00000000e+00, -5.83496094e-02,  2.92968750e-01,  2.04101562e-01,
       -3.47656250e-01, -4.22363281e-02, -1.76757812e-01,  2.08007812e-01,
       -7.56835938e-03,  2.83203125e-01, -1.85546875e-01,  2.71484375e-01,
       -1.69921875e-01,  4.83398438e-02, -1.74804688e-01, -9.52148438e-02,
        2.62451172e-02, -1.73828125e-01, -1.44531250e-01, -3.66210938e-02,
       -6.25000000e-02,  5.00488281e-02,  5.78613281e-02, -4.63867188e-02,
       -2.05078125e-02,  1.33789062e-01, -2.55859375e-01, -1.36718750e-01,
        1.15234375e-01,  9.17968750e-02, -3.51562500e-01, -2.31445312e-01,
       -6.29882812e-02, -1.81640625e-01,  1.79687500e-01, -1.80664062e-01,
       -2.50000000e-01,  7.47070312e-02,  3.00781250e-01,  1.64062500e-01,
       -2.77343750e-01, -2.51953125e-01,  1.46484375e-01,  2.09960938e-01,
       -1.88476562e-01, -4.49218750e-02, -2.08984375e-01, -1.01318359e-02,
        2.99072266e-02, -1.48437500e-01,  2.73437500e-01,  4.05273438e-02,
       -2.92968750e-01,  

In [312]:
len(vocab)

5177

In [313]:
len(word2idx['guitar'])

402

In [314]:
tag2idx = {t: i for i, t in enumerate(tag)}
idx2tag = {v: k for k, v in iteritems(tag2idx)}

In [315]:
def prepare_sequence(seq, to_idx):
    
    idxs = [to_idx[w] for w in seq]
    idxs = np.array(idxs)
    output_idxs = torch.from_numpy(idxs)
    output_idxs = output_idxs.type(torch.FloatTensor).cuda()
    
    return output_idxs

In [316]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        ''' Initialize the layers of this model.'''
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        
        #LAYER1 : EMBEDDING LAYER
        # embedding layer that turns words into a vector of a specified size
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        #LAYER2 : LSTM
        # the LSTM takes embedded word vectors (of a specified size) as inputs and outputs hidden states of size hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        #LAYER3 : DENSE
        # the linear layer that maps the hidden state output dimension to the number of tags we want as output, tagset_size (in this case this is 3 tags)
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)
        
        # initialize the hidden state (see code below)
        self.hidden = self.init_hidden()

        
    def init_hidden(self):

        # The axes dimensions are (n_layers, batch_size, hidden_dim)
        return(torch.randn(2, 1, self.hidden_dim).cuda(),
            torch.randn(2, 1, self.hidden_dim).cuda())
        
        #return (torch.zeros(2, 1, self.hidden_dim).cuda(),
        #       torch.zeros(2, 1, self.hidden_dim).cuda())
    
    def forward(self, sentence):
        
        embeds = sentence
        
        lstm_out, self.hidden = self.lstm((embeds.view(len(sentence), 1, -1)), self.hidden)
                
        # get the scores for the most likely tag for a word
        tag_scores = self.hidden2tag(lstm_out.view(len(sentence), -1))

        return tag_scores

In [317]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
torch.cuda.set_device(3)
print(torch.cuda.current_device())

True
3
3


In [318]:
torch.manual_seed(12345)

In [319]:
# the embedding dimension defines the size of our word vectors
# for our simple vocabulary and training set, we will keep these small
EMBEDDING_DIM = 402
HIDDEN_DIM = 120

# instantiate our model
model1 = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), len(tag2idx))
model1.cuda()

# define our loss and optimizer
loss_function =  nn.CrossEntropyLoss()  #nn.NLLLoss()   
optimizer = optim.SGD(model1.parameters(), lr=0.1)       ##SGD


In [320]:
train_x

0      [i, used, to, get, terrible, headaches, from, ...
1      [peters, s, pain, and, symptoms, were, caused,...
2      [in, the, years, since, the, adm, scandal, of,...
3      [a, first, revolution, was, triggered, by, the...
4      [when, they, took, the, floor, against, a, tea...
                             ...                        
797    [using, solar, electricity, instead, of, conve...
798    [fatigue, corrosion, and, stress, corrosion, a...
799    [the, electromagnetic, em, radiation, from, th...
800    [it, gets, a, little, bit, chicken, first, or,...
801    [the, microphone, converts, sound, into, an, e...
Name: token, Length: 802, dtype: object

In [321]:
test_x

0      [many, adults, retain, scars, from, acne, brea...
1      [we, estimate, a, wind, speed, associated, wit...
2      [outbreaks, caused, by, the, oral, vaccine, s,...
3      [production, and, investigation, of, such, a, ...
4      [the, drugs, he, sold, had, caused, the, overd...
                             ...                        
196    [the, real, possibility, of, total, engulfment...
197    [we, find, evidence, that, ernst, suffered, a,...
198    [the, influx, caused, a, further, drain, on, t...
199    [ngu, nongonococcal, urethritis, is, an, infec...
200    [eye, discomfort, from, this, staring, effect,...
Name: token, Length: 201, dtype: object

In [322]:
training_input_data = []
for i in range(len(train_x)):
    sen = train_x[i]
    tag = train_y[i]
    sen_t = prepare_sequence(sen, word2idx)
    tag_t = prepare_sequence(tag, tag2idx)
    training_input_data.append((sen_t, tag_t))
    
    
testing_input_data = []
for i in range(len(test_x)):
    sen = test_x[i]
    tag = test_y[i]
    sen_t = prepare_sequence(sen, word2idx)
    tag_t = prepare_sequence(tag, tag2idx)
    testing_input_data.append((sen_t, tag_t))
    
    

In [323]:
len(train_x[0])

20

In [324]:
len(training_input_data[0][0])

20

In [325]:
n_epochs = 15
score_save = []

start_time = time.time()
for epoch in range(n_epochs):
    
    epoch_loss = 0.0
    count=0
    
    for sentence, tags in training_input_data:
        if len(sentence)>0:
            count+=1
            
            model1.zero_grad()

            # zero the hidden state of the LSTM, this detaches it from its history
            model1.hidden = model1.init_hidden()

           
            tags = tags.type(torch.LongTensor)  #.cuda()
            
            tag_scores = model1(sentence)
            
            # Loss
            tag_scores = tag_scores.type(torch.FloatTensor)  #.cuda()
            #targets = targets.type(torch.LongTensor).cuda()
            if epoch==n_epochs-1:
                score_save.append(tag_scores.tolist())
            
            loss = loss_function(tag_scores, tags)
            #print(loss)
            epoch_loss += loss.item()
            loss.backward()

            # update the model parameters with optimizer.step()
            optimizer.step()
        
    
    print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(train_x)))
    print("--- %s seconds ---" % (time.time() - start_time))

print("Total time taken : ")
print("--- %s seconds ---" % (time.time() - start_time))

Epoch: 1, loss: 0.37979
--- 4.5094990730285645 seconds ---
Epoch: 2, loss: 0.29686
--- 8.843169927597046 seconds ---
Epoch: 3, loss: 0.26581
--- 13.295043468475342 seconds ---
Epoch: 4, loss: 0.24212
--- 17.726362705230713 seconds ---
Epoch: 5, loss: 0.22042
--- 22.18460488319397 seconds ---
Epoch: 6, loss: 0.20351
--- 26.614367246627808 seconds ---
Epoch: 7, loss: 0.18584
--- 31.06912660598755 seconds ---
Epoch: 8, loss: 0.17068
--- 35.50715899467468 seconds ---
Epoch: 9, loss: 0.15479
--- 39.9329469203949 seconds ---
Epoch: 10, loss: 0.14721
--- 44.350550413131714 seconds ---
Epoch: 11, loss: 0.13186
--- 48.772984743118286 seconds ---
Epoch: 12, loss: 0.12377
--- 53.227378606796265 seconds ---
Epoch: 13, loss: 0.10995
--- 57.65360951423645 seconds ---
Epoch: 14, loss: 0.10053
--- 62.07853031158447 seconds ---
Epoch: 15, loss: 0.09381
--- 66.5236988067627 seconds ---
Total time taken : 
--- 66.52557945251465 seconds ---


In [294]:
#Evaluation for training data

pred_tag = []
actual_tag = []
for sen, tag in training_input_data:
    tag_scores = model1(sen)
    _, predicted_tags = torch.max(tag_scores, 1)
    pred = predicted_tags.cpu().tolist()
    pred = [idx2tag[idx] for idx in pred]
    pred_tag.append(pred)
    tag = tag.cpu().tolist()
    act = [idx2tag[idx] for idx in tag]
    actual_tag.append(act)


In [295]:
pred_tag_f = [item for sublist in pred_tag for item in sublist]
actual_tag_f = [item for sublist in actual_tag for item in sublist]

In [296]:
from sklearn.metrics import classification_report

label_names = ['O', 'C', 'E']

print(classification_report(actual_tag_f, pred_tag_f,target_names=label_names))

              precision    recall  f1-score   support

           O       0.93      0.61      0.74       890
           C       0.93      0.72      0.81       887
           E       0.96      1.00      0.98     13185

    accuracy                           0.96     14962
   macro avg       0.94      0.77      0.84     14962
weighted avg       0.96      0.96      0.95     14962



In [297]:
#Evaluation for testing data

pred_tag = []
actual_tag = []
for sen, tag in testing_input_data:
    tag_scores = model1(sen)
    _, predicted_tags = torch.max(tag_scores, 1)
    pred = predicted_tags.cpu().tolist()
    pred = [idx2tag[idx] for idx in pred]
    pred_tag.append(pred)
    tag = tag.cpu().tolist()
    act = [idx2tag[idx] for idx in tag]
    actual_tag.append(act)


In [298]:
pred_tag_f = [item for sublist in pred_tag for item in sublist]
actual_tag_f = [item for sublist in actual_tag for item in sublist]

In [299]:
from sklearn.metrics import classification_report

label_names = ['O', 'C', 'E']

print(classification_report(actual_tag_f, pred_tag_f,target_names=label_names))

              precision    recall  f1-score   support

           O       0.59      0.33      0.43       224
           C       0.69      0.50      0.58       217
           E       0.94      0.98      0.96      3516

    accuracy                           0.91      3957
   macro avg       0.74      0.60      0.65      3957
weighted avg       0.90      0.91      0.91      3957



In [37]:
def clean_x(sen):
    sen = re.sub('[^a-zA-Z0-9 \n\.]', ' ', sen)
    sen = sen.replace('.', ' ')
    sen = sen.lower()
    sen = sen.split()
    return sen

def clean_y(rel):
    cause = rel[0]
    cause = re.sub('[^a-zA-Z0-9 \n\.]', ' ', cause)
    cause = cause.lower().split()
    effect = rel[1]
    effect = re.sub('[^a-zA-Z0-9 \n\.]', ' ', effect)
    effect = effect.lower().split()

    return ((cause, effect))

In [38]:
#TEST WITH ANY INPUT SENTENCE


def test_with_sen(input_sen):
    input_sen = clean_x(input_sen)
    
    inputs = prepare_sequence(input_sen, word2idx)

    tag_scores = model1(inputs)


    _, predicted_tags = torch.max(tag_scores, 1)

    print("Input sentence: ", input_sen)
    for i in range(0,len(input_sen)):
        t = ''
        if predicted_tags[i].item()==0:
            t = 'Other'
        elif predicted_tags[i].item()==1:
            t = 'Cause'
        elif predicted_tags[i].item()==2:
            t = 'Effect'
        #elif predicted_tags[i].item==3:
        #    t = 'B-Effect'
        #else:
        #    t = 'I-Effect'
        print(predicted_tags[i].item())
        print(input_sen[i] , ': ', t )
    return
    

In [39]:
test_sen = 'Smoking causes cancer'
test_with_sen(test_sen)

Input sentence:  ['smoking', 'causes', 'cancer']
1
smoking :  Cause
0
causes :  Other
0
cancer :  Other


In [40]:
a = ' '.join(test_x[46])

print(a)
test_with_sen(a)

there are many types and causes of headaches from tension headaches due to stress to migraines triggered by certain foods
Input sentence:  ['there', 'are', 'many', 'types', 'and', 'causes', 'of', 'headaches', 'from', 'tension', 'headaches', 'due', 'to', 'stress', 'to', 'migraines', 'triggered', 'by', 'certain', 'foods']
0
there :  Other
0
are :  Other
0
many :  Other
0
types :  Other
0
and :  Other
0
causes :  Other
0
of :  Other
2
headaches :  Effect
0
from :  Other
0
tension :  Other
2
headaches :  Effect
0
due :  Other
0
to :  Other
1
stress :  Cause
0
to :  Other
2
migraines :  Effect
0
triggered :  Other
0
by :  Other
0
certain :  Other
1
foods :  Cause
